# How Local Income Inequality Shapes Public Concern

This notebook converts the following R code into Python. It demonstrates data preparation, merging survey data with administrative indicators, and multi-level regression modeling.

In [24]:
import pandas as pd

# 1. Load survey data
survey_url = (
    "https://raw.githubusercontent.com/"
    "kmazurek95/Local-Inequality-and-Redistribution-Attitudes-A-Quantitative-Analysis/"
    "main/AISSR%20Internship/Data/score.dta"
)
survey_raw = pd.read_stata(survey_url)

# 2. Build the three merge keys from Buurtcode
def make_buurt_id(x):
    if pd.isna(x):
        return pd.NA
    return str(int(x)).zfill(8)

survey_raw["buurt_id"]    = survey_raw["Buurtcode"].apply(make_buurt_id)
survey_raw["wijk_id"]     = survey_raw["buurt_id"].astype(str).str[:6]
survey_raw["gemeente_id"] = survey_raw["buurt_id"].astype(str).str[:4]

# 3. Subset & rename survey vars
keep = [
    "a27_1","a27_2","a27_3","b01","b02","b03","b04","b05","b06",
    "b07","b08","b09","b10","b11","b12_1","b13",
    "b14_1","b14_2","b14_3","b14_4","b14_5",
    "b15","b16","b17","b18","b20","b21","b22",
    "weegfac","Buurtcode","buurt_id","wijk_id","gemeente_id","respnr"
]
rename_map = {
    "a27_1":"gov_int","a27_2":"red_inc_diff","a27_3":"union_pref",
    "b01":"sex","b02":"birth_year","b03":"educlvl","b04":"educyrs",
    "b05":"voted","b06":"vote_choice","b07":"work_status",
    "b08":"has_worked","b09":"work_type","b10":"org_type",
    "b11":"leader_resp","b12_1":"leader_resp_n","b13":"job_desc",
    "b14_1":"home_owner","b14_2":"other_real_estate","b14_3":"savings",
    "b14_4":"stocks","b14_5":"no_owner","b15":"relig_yn",
    "b16":"religiosity","b17":"relig_attend","b18":"born_in_nl",
    "b20":"father_dutch","b21":"mother_dutch","b22":"weight",
    "weegfac":"geocode","respnr":"respondent_id"
}
survey = survey_raw[keep].rename(columns=rename_map)

# 4. Load admin data
admin_url = (
    "https://raw.githubusercontent.com/"
    "kmazurek95/Local-Inequality-and-Redistribution-Attitudes-A-Quantitative-Analysis/"
    "main/AISSR%20Internship/Data/indicators_buurt_wijk_gemeente.csv"
)
admin = pd.read_csv(admin_url, na_values=".")
admin = admin.drop(columns=[
    "Codering_3",
    "IndelingswijzigingWijkenEnBuurten_4",
    "Marokko_19",
    "Turkije_22"
])

# 5. Rename first 18 cols
new_cols = [
    "id","code","municipality","region_type","pop_total","pop_over_65",
    "pop_west","pop_nonwest","pop_dens","avg_home_value","avg_inc_recip",
    "avg_inc_pers","perc_low40_pers","perc_high20_pers",
    "perc_low40_hh","perc_high20_hh","perc_low_inc_hh","perc_soc_min_hh"
]
admin.columns = new_cols + list(admin.columns[18:])

# 6. Normalize region_type & create code_str
admin["region_type"] = admin["region_type"].astype(str).str.strip().str.lower()
admin["code_str"]    = admin["code"].astype(str).str.replace(r"\.0$","",regex=True)

# Helper to prefix only non-key columns
def prefix_other_cols(df, key, prefix):
    cols = df.columns.tolist()
    rename = {c: (c if c==key else prefix + c) for c in cols}
    return df.rename(columns=rename)

# 7. Build Buurt table
buurt = admin[admin.region_type=="buurt"].copy()
buurt["buurt_id"] = buurt["code_str"].str[2:]        # drop "BU"
buurt = prefix_other_cols(buurt, "buurt_id", "b_")

# 8. Build Wijk table
wijk = admin[admin.region_type=="wijk"].copy()
wijk["wijk_id"] = wijk["code_str"].str[2:]           # drop "WK"
wijk = prefix_other_cols(wijk, "wijk_id", "w_")

# 9. Build Gemeente table
gemeente = admin[admin.region_type=="gemeente"].copy()
gemeente["gemeente_id"] = gemeente["code_str"].str[2:]  # drop "GM"
gemeente = prefix_other_cols(gemeente, "gemeente_id", "g_")

# 10. Merge with indicators
check = (
    survey
      .merge(buurt,    on="buurt_id",    how="left", indicator="buurt_match")
      .merge(wijk,     on="wijk_id",     how="left", indicator="wijk_match")
      .merge(gemeente, on="gemeente_id", how="left", indicator="gemeente_match")
)

print("=== Merge Indicators ===")
print("Buurt:   ",    check["buurt_match"].value_counts().to_dict())
print("Wijk:    ",    check["wijk_match"].value_counts().to_dict())
print("Gemeente:",    check["gemeente_match"].value_counts().to_dict())


=== Merge Indicators ===
Buurt:    {'both': 7160, 'left_only': 853, 'right_only': 0}
Wijk:     {'left_only': 8013, 'right_only': 0, 'both': 0}
Gemeente: {'left_only': 8013, 'right_only': 0, 'both': 0}


### Pulling Score Data from GitHub

Assuming your repository is public, you can use the raw URL to load the data directly into pandas. For example, if your score data file is named `score.dta` and is located in the folder `AISSR Internship/Data/` of your repository, the raw URL looks similar to the following:

In [9]:
# Import the pandas library
import pandas as pd
# Load required packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
import statsmodels.api as sm


# Now load the score data (survey data) from GitHub using the defined URL
score_url = "https://raw.githubusercontent.com/kmazurek95/Local-Inequality-and-Redistribution-Attitudes-A-Quantitative-Analysis/main/AISSR%20Internship/Data/score.dta"
score = pd.read_stata(score_url)
print(score.head())  # display the first few rows


   respnr s01 a01                 a02_1                   a02_2  \
0     1.0  Ja   2                     3                       2   
1     2.0  Ja   3                     6                       4   
2     3.0  Ja   5  Helemaal mee eens  7    Helemaal mee eens  7   
3     4.0  Ja   2                     3                       3   
4     5.0  Ja   5                     4  Helemaal mee oneens  1   

                                a03         a04 a05                a06  \
0                                 3           3   2  1  Helemaal niet   
1                                 5           5   4                  2   
2                                 3           2   2  1  Helemaal niet   
3                                 5           4   5                  3   
4  1 - Helemaal niet geïnteresseerd  1  Altijd   3  1  Helemaal niet   

                 a07  ... SEGMENTNAME   Stemgedrag_2017         Stedelijkheid  \
0  1  Helemaal niet  ...          S1      Niet-stemmer  zeer sterk ste

## Geo Code Validation Summary

This output shows the results of your geo code validation checks after processing the survey data:

---

### 🔍 Missing `buurt_id` / `wijk_id` / `gemeente_id`

- **26 missing values** for each of these columns.
- This means **26 survey records have a missing or invalid `Buurtcode`**, so their derived geo codes could not be created.
- This is expected if:
  - Some respondents did not provide a valid buurt code.
  - There were errors in the source data.

---

### 📏 `buurt_id` Lengths

- ✅ 7,987 codes have **length 8** (correct — `buurt_id` should be 8 digits).
- ⚠️ 26 codes have **length 4** (these correspond to the missing/invalid cases above).

---

### 📏 `wijk_id` Lengths

- ✅ 7,987 codes have **length 6** (correct).
- ⚠️ 26 codes have **length 4** (again, these are the missing/invalid cases).

---

### 📏 `gemeente_id` Lengths

- ✅ All **8,013 codes** have **length 4** (correct).

---

### ✅ Sample Hierarchy Checks

- The **first 6 digits** of `buurt_id` match `wijk_id`.
- The **first 4 digits** match `gemeente_id`.
- This confirms the **hierarchical structure is preserved** for valid codes.

---

### 💬 Is This What We Desire?

**Yes, this is mostly correct.**

- The majority of geo codes are **formatted correctly**, and the **hierarchy is consistent**.
- The only issue is the **26 missing/invalid codes**, which:
  - May warrant investigation or exclusion from further analysis.
  - Are typical in survey data if some respondents skip or mistype codes.
  - Should be checked against source data **if every respondent is expected to provide a valid geo code**.

---


In [11]:
# 2. Build the merge keys — handle NaNs gracefully
def make_buurt_id(x):
    if pd.isna(x):
        return pd.NA          # leave as missing
    # cast to int, then to string, then pad to width 8
    return str(int(x)).zfill(8)

score['buurt_id']      = score['Buurtcode'].apply(make_buurt_id)
score['wijk_id']       = score['buurt_id'].astype(str).str[:6]
score['gemeente_id']   = score['buurt_id'].astype(str).str[:4]

# Quick check
print(score[['Buurtcode','buurt_id','wijk_id','gemeente_id']].head(10))

    Buurtcode  buurt_id wijk_id gemeente_id
0   6220102.0  06220102  062201        0622
1   1144200.0  01144200  011442        0114
2   2210104.0  02210104  022101        0221
3   8580002.0  08580002  085800        0858
4   2680321.0  02680321  026803        0268
5    340201.0  00340201  003402        0034
6  19300502.0  19300502  193005        1930
7   5183284.0  05183284  051832        0518
8   4000408.0  04000408  040004        0400
9   3080007.0  03080007  030800        0308


In [12]:
# Subset and rename variables for clarity
# 3. Subset & rename survey variables
keep = [
    "a27_1","a27_2","a27_3","b01","b02","b03","b04","b05","b06",
    "b07","b08","b09","b10","b11","b12_1","b13",
    "b14_1","b14_2","b14_3","b14_4","b14_5",
    "b15","b16","b17","b18","b20","b21","b22",
    "weegfac","Buurtcode","buurt_id","wijk_id","gemeente_id","respnr"
]

rename_map = {
    'a27_1': 'gov_intervention_attitude',      # Attitude: less government intervention is better
    'a27_2': 'income_redistribution_attitude', # Attitude: government should reduce income differences
    'a27_3': 'union_support_attitude',         # Attitude: need for strong trade unions
    'b01': 'sex',                              # Sex (Male/Female/Other)
    'b02': 'birth_year',                       # Year of birth
    'b03': 'education_level',                  # Highest completed education level
    'b04': 'education_years',                  # Years of education completed
    'b05': 'employment_status',                # Current employment status
    'b06': 'has_worked',                       # Ever done paid work
    'b07': 'job_type',                         # Employee/self-employed/family business
    'b08': 'organization_type',                # Type of organization worked for
    'b09': 'supervisory_role',                 # Supervisory responsibility
    'b10': 'num_supervised',                   # Number of people supervised
    'b11': 'employment_description',           # Employment description
    'b12_1': 'owns_real_estate',               # Owns real estate
    'b13': 'religion_affiliation',             # Belongs to a religion/denomination
    'b14_1': 'religion_type',                  # Type of religion
    'b14_2': 'owns_additional_real_estate',    # Owns further real estate
    'b14_3': 'owns_savings_account',           # Owns savings account
    'b14_4': 'owns_stocks',                    # Owns stocks/shares
    'b14_5': 'owns_none',                      # Owns none of the above
    'b15': 'religiosity',                      # How religious
    'b16': 'religious_attendance',             # Frequency of attending religious services
    'b17': 'prayer_frequency',                 # Frequency of prayer
    'b18': 'born_in_country',                  # Born in country
    'b20': 'father_born_in_country',           # Father born in country
    'b21': 'mother_born_in_country',           # Mother born in country
    'b22': 'voted_last_election',              # Voted in last national election
    'weegfac': 'survey_weight_factor',         # Survey weight factor
    'Buurtcode': 'original_buurt_code',        # Original buurt code
    'buurt_id': 'buurt_id',                    # Standardized buurt code
    'wijk_id': 'wijk_id',                      # Standardized wijk code
    'gemeente_id': 'gemeente_id',              # Standardized gemeente code
    'respnr': 'respondent_id'                  # Respondent ID
}

survey = score[keep].rename(columns=rename_map)


In [16]:
# 4. Load and prep administrative data
admin_url = (
    "https://raw.githubusercontent.com/"
    "kmazurek95/Local-Inequality-and-Redistribution-Attitudes-A-Quantitative-Analysis/"
    "main/AISSR%20Internship/Data/indicators_buurt_wijk_gemeente.csv"
)
admin = pd.read_csv(admin_url, na_values=".")
admin = admin.drop(columns=[
    "Codering_3",
    "IndelingswijzigingWijkenEnBuurten_4",
    "Marokko_19",
    "Turkije_22"
])
# rename first 18 columns
new_names = [
    "id","code","municipality","region_type","pop_total","pop_over_65",
    "pop_west","pop_nonwest","pop_dens","avg_home_value","avg_inc_recip",
    "avg_inc_pers","perc_low40_pers","perc_high20_pers",
    "perc_low40_hh","perc_high20_hh","perc_low_inc_hh","perc_soc_min_hh"
]
admin.columns = new_names + list(admin.columns[18:])

# split & prefix
buurt    = admin[admin.region_type=="Buurt"].copy().add_prefix("b_")
wijk     = admin[admin.region_type=="Wijk"].copy().add_prefix("w_")
gemeente = admin[admin.region_type=="Gemeente"].copy().add_prefix("g_")

buurt["buurt_id"]       = buurt["b_code"].str[2:]
wijk["wijk_id"]         = wijk["w_code"].str[2:]
gemeente["gemeente_id"] = gemeente["g_code"].str[2:]

# 5. Merge everything
merged = (
    survey
    .merge(buurt,    on="buurt_id",    how="left")
    .merge(wijk,     on="wijk_id",     how="left")
    .merge(gemeente, on="gemeente_id", how="left")
)

print("Merged rows:", merged.shape[0])
# Save the merged DataFrame to the Downloads folder
merged.to_csv(r"C:\Users\kaleb\Downloads\complete_merge.csv", index=False)

Merged rows: 8013


In [14]:
# 9. Sequentially merge survey + buurt + wijk + gemeente
merged = (
    survey
    .merge(buurt,    on="buurt_id",     how="left")
    .merge(wijk,     on="wijk_id",      how="left")
    .merge(gemeente, on="gemeente_id",  how="left")
)

# 10. Inspect & save
print("Merged shape:", merged.shape)
print( merged.filter(regex="buurt_id|b_pop_total|w_pop_total|g_pop_total").head() )

# Save the merged DataFrame to the Downloads folder
merged.to_csv(r"C:\Users\kaleb\Downloads\complete_merge.csv", index=False)

Merged shape: (8013, 88)
   buurt_id  b_pop_total  w_pop_total  g_pop_total
0  06220102          NaN          NaN          NaN
1  01144200          NaN          NaN          NaN
2  02210104          NaN          NaN          NaN
3  08580002          NaN          NaN          NaN
4  02680321          NaN          NaN          NaN


In [ ]:
# 1. Find survey buurt_ids not found in admin data
missing_buurt_ids = set(score_final['buurt_id']) - set(ind_bu['buurt_id'])
print("Survey buurt_ids not found in admin data:", missing_buurt_ids)

# 2. Find survey wijk_ids not found in admin data
missing_wijk_ids = set(score_final['wijk_id']) - set(ind_wi['wijk_id'])
print("Survey wijk_ids not found in admin data:", missing_wijk_ids)

# 3. Find survey gemeente_ids not found in admin data
missing_gemeente_ids = set(score_final['gemeente_id']) - set(ind_ge['gemeente_id'])
print("Survey gemeente_ids not found in admin data:", missing_gemeente_ids)


In [ ]:
# Show a sample of unmatched buurt_ids and their survey rows
unmatched_buurt = score_final[score_final['buurt_id'].isin(missing_buurt_ids)]
# Show a sample of unmatched buurt_ids and their survey rows
print(unmatched_buurt[['buurt_id', 'wijk_id', 'gemeente_id', 'Buurtcode']].head(10))

# See if any unmatched codes are present in admin data with a different format
for code in list(missing_buurt_ids)[:10]:
    print(ind_bu[ind_bu['buurt_id'].str.contains(str(code), na=False)])

In [ ]:
# 4. Merge Survey and Administrative Data

import os

# Create the directory if it doesn't exist
os.makedirs("./data", exist_ok=True)

# --- Review the Merge: Check joining keys before merging ---
# Check unique keys from the survey data
print("Unique survey 'buurt_id':", score_final['buurt_id'].unique())
print("Unique survey 'wijk_id':", score_final['wijk_id'].unique())
print("Unique survey 'gemeente_id':", score_final['gemeente_id'].unique())

# Check unique keys from the administrative data (example for Buurt-level)
print("Unique administrative 'buurt_id':", ind_bu['buurt_id'].unique())

# --- Merge the datasets ---
merge_temp1 = pd.merge(score_final, ind_bu, on="buurt_id", how="left")
merge_temp2 = pd.merge(merge_temp1, ind_wi, on="wijk_id", how="left")
complete_merge = pd.merge(merge_temp2, ind_ge, on="gemeente_id", how="left")

# --- Review the merged data ---
# Verify that key columns in the merged data have non-missing values
print("Non-missing count for 'buurt_id' in merged data:", complete_merge['buurt_id'].notna().sum())
print("Non-missing count for 'wijk_id' in merged data:", complete_merge['wijk_id'].notna().sum())
print("Non-missing count for 'gemeente_id' in merged data:", complete_merge['gemeente_id'].notna().sum())

# Save the merged data to a local folder; add 'data/' to your .gitignore if this file is generated automatically.
output_path = "./data/complete_merge1.csv"
complete_merge.to_csv(output_path, index=False)
print(f"Merged data saved to {output_path}")

In [ ]:
#print("Merged data preview:")
complete_merge.head()  # Display the first few rows of the merged data

In [ ]:
# Print sample IDs and their lengths for wijk_id
print("Sample survey wijk_ids:", list(score_final['wijk_id'].unique())[:10])
print("Sample admin wijk_ids:", list(ind_wi['wijk_id'].unique())[:10])
print("Survey wijk_id dtype:", score_final['wijk_id'].dtype, "Admin wijk_id dtype:", ind_wi['wijk_id'].dtype)
print("Survey wijk_id lengths:", score_final['wijk_id'].str.len().value_counts())
print("Admin wijk_id lengths:", ind_wi['wijk_id'].str.len().value_counts())

# Repeat for gemeente_id
print("Sample survey gemeente_ids:", list(score_final['gemeente_id'].unique())[:10])
print("Sample admin gemeente_ids:", list(ind_ge['gemeente_id'].unique())[:10])
print("Survey gemeente_id dtype:", score_final['gemeente_id'].dtype, "Admin gemeente_id dtype:", ind_ge['gemeente_id'].dtype)
print("Survey gemeente_id lengths:", score_final['gemeente_id'].str.len().value_counts())
print("Admin gemeente_id lengths:", ind_ge['gemeente_id'].str.len().value_counts())

In [ ]:
print("Sample survey buurt_ids:", list(score_final['buurt_id'].unique())[:10])
print("Sample admin buurt_ids:", list(ind_bu['buurt_id'].unique())[:10])

print(score_final['buurt_id'].dtype, ind_bu['buurt_id'].dtype)
print(score_final['buurt_id'].str.len().value_counts())
print(ind_bu['buurt_id'].str.len().value_counts())

In [ ]:
# Examine the final merged DataFrame

# Check the shape of the merged DataFrame
print("Shape of complete_merge:", complete_merge.shape)

# Check for missing values in the merged DataFrame
missing_values = complete_merge.isna().sum()
print("\nMissing values per column:")
print(missing_values[missing_values > 0])

# Check the first few rows of the merged DataFrame
print("\nFirst few rows of complete_merge:")
print(complete_merge.head())

# Check the unique values of key columns to ensure the merge worked correctly
print("\nUnique values in 'buurt_id':", complete_merge['buurt_id'].unique())
print("\nUnique values in 'wijk_id':", complete_merge['wijk_id'].unique())
print("\nUnique values in 'gemeente_id':", complete_merge['gemeente_id'].unique())

# Check the presence of administrative variables
admin_vars = [col for col in complete_merge.columns if col.startswith(('b_', 'w_', 'g_'))]
print("\nPresence of administrative variables:")
print(complete_merge[admin_vars].notna().sum())

In [ ]:
# Check which buurt_ids in your survey data are NOT in the admin data
missing_buurt_ids = set(score_final['buurt_id']) - set(ind_bu['buurt_id'])
print("Survey buurt_ids not found in admin data:", missing_buurt_ids)

# Do the same for wijk_id and gemeente_id
missing_wijk_ids = set(score_final['wijk_id']) - set(ind_wi['wijk_id'])
print("Survey wijk_ids not found in admin data:", missing_wijk_ids)

missing_gemeente_ids = set(score_final['gemeente_id']) - set(ind_ge['gemeente_id'])
print("Survey gemeente_ids not found in admin data:", missing_gemeente_ids)

In [ ]:
# count the number of rows and columns in the merged DataFrame
print("\nNumber of rows in the merged DataFrame:", complete_merge.shape[0])
print("Number of columns in the merged DataFrame:", complete_merge.shape[1])

# count the number of missing observations for each of the administrative variables
missing_admin_vars = complete_merge[admin_vars].isna().sum()
print("\nMissing observations for administrative variables:")
print(missing_admin_vars[missing_admin_vars > 0])
print(missing_admin_vars[missing_admin_vars > 0])

In [ ]:
# 5. Data Transformation & Variable Recoding
data = pd.read_csv("./data/complete_merge1.csv")

# Convert relevant columns to numeric if they're not already
cols_to_convert = ['red_inc_diff', 'gov_int', 'union_pref']
for col in cols_to_convert:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Filter out unwanted rows
data = data[(data['red_inc_diff'] != 8) & (data['gov_int'] != 8) & (data['union_pref'] != 8)]

# Compute DV_one_to_zero safely now that the columns are numeric
data['DV_one_to_zero'] = 100 * (data['red_inc_diff'] - 1) / (7 - 1)

# Recode sex as a factor with descriptive labels
data['sex'] = data['sex'].apply(lambda x: "Male" if x == 1 else ("Female" if x == 2 else "Other"))

# Create and standardize age (assuming survey conducted in 2017) and education
data['age'] = 2017 - data['birth_year']
scaler = StandardScaler()
data['age'] = scaler.fit_transform(data[['age']])
data['education'] = scaler.fit_transform(data[['educyrs']])

# Standardize selected neighborhood-level administrative variables
cols_to_scale = [
    'b_perc_low40_hh', 'b_pop_total', 'b_pop_over_65', 'b_pop_nonwest',
    'b_avg_inc_recip', 'b_perc_low_inc_hh', 'b_pop_dens', 'b_perc_soc_min_hh'
]
for col in cols_to_scale:
    if col in data.columns:
        data[col] = scaler.fit_transform(data[[col]])

In [ ]:
# 6. Multi-Level Regression Analysis in Python

# Define admin variable columns by selecting those with prefixes ('b_', 'w_', 'g_').
admin_vars = [col for col in complete_merge.columns if col.startswith(('b_', 'w_', 'g_'))]

# (1) Count the number of present admin variables for each observation.
complete_merge['admin_presence_count'] = complete_merge[admin_vars].notna().sum(axis=1)
print(complete_merge[['buurt_id', 'admin_presence_count']].head())

# (2) List which admin variables are present for each observation.
def present_admin_vars(row):
    return [col for col in admin_vars if pd.notna(row[col])]

complete_merge['present_admin_vars'] = complete_merge.apply(present_admin_vars, axis=1)
print(complete_merge[['buurt_id', 'present_admin_vars']].head())

# Create a subset of variables and drop rows with missing data
data_two_levels = data[[
    'DV_one_to_zero', 'age', 'sex', 'education',
    'b_perc_low40_hh', 'b_pop_total', 'b_pop_over_65', 'b_pop_nonwest',
    'b_avg_inc_recip', 'b_perc_low_inc_hh', 'b_pop_dens', 'b_perc_soc_min_hh',
    'buurt_id'
]].dropna()

# Check the shape of the data
print("Shape of data_two_levels:")
print(data_two_levels.shape)

# Check for missing values
print("Shape of data before filtering:", data.shape)
print("Unique values for red_inc_diff:", data['red_inc_diff'].unique())
print("Unique values for gov_int:", data['gov_int'].unique())
print("Unique values for union_pref:", data['union_pref'].unique())

# Check the columns of the data
print(data.columns.tolist())

#  Check for missing values in the required columns
required_cols = [
    'DV_one_to_zero', 'age', 'sex', 'education',
    'b_perc_low40_hh', 'b_pop_total', 'b_pop_over_65', 'b_pop_nonwest',
    'b_avg_inc_recip', 'b_perc_low_inc_hh', 'b_pop_dens', 'b_perc_soc_min_hh',
    'buurt_id'
]
print(data[required_cols].isna().sum())

# Ensure that 'sex' is treated as a categorical variable
data_two_levels['sex'] = data_two_levels['sex'].astype('category')

import statsmodels.formula.api as smf

# -- Model 0: Empty Model (Intercept Only) --
m0 = smf.mixedlm("DV_one_to_zero ~ 1", data_two_levels, groups=data_two_levels["buurt_id"]).fit()
print("Model 0: Empty Model (Intercept Only)")
print(m0.summary())

# -- Model 1: Add Key Neighborhood Variable --
m1 = smf.mixedlm("DV_one_to_zero ~ b_perc_low40_hh", data_two_levels, groups=data_two_levels["buurt_id"]).fit()
print("\nModel 1: Add Key Neighborhood Variable")
print(m1.summary())

# Likelihood ratio test between Model 0 and Model 1
lrtest_01 = m0.compare_lr_test(m1)
print("\nLikelihood Ratio Test (Model 0 vs Model 1):")
print(f"Chi2: {lrtest_01[0]:.4f}, p-value: {lrtest_01[1]:.4f}, df: {lrtest_01[2]}")

# -- Model 2: Add Individual-Level Controls --
# Using C(sex) to indicate a categorical variable in the formula
m2 = smf.mixedlm("DV_one_to_zero ~ b_perc_low40_hh + age + C(sex) + education",
                 data_two_levels, groups=data_two_levels["buurt_id"]).fit()
print("\nModel 2: Add Individual-Level Controls")
print(m2.summary())

lrtest_12 = m1.compare_lr_test(m2)
print("\nLikelihood Ratio Test (Model 1 vs Model 2):")
print(f"Chi2: {lrtest_12[0]:.4f}, p-value: {lrtest_12[1]:.4f}, df: {lrtest_12[2]}")

# -- Model 3: Add Additional Neighborhood-Level Controls --
m3 = smf.mixedlm("""DV_one_to_zero ~ b_perc_low40_hh + age + C(sex) + education +
                    b_pop_dens + b_pop_over_65 + b_pop_nonwest +
                    b_perc_low_inc_hh + b_perc_soc_min_hh""",
                 data_two_levels, groups=data_two_levels["buurt_id"]).fit()
print("\nModel 3: Add Additional Neighborhood-Level Controls")
print(m3.summary())

lrtest_23 = m2.compare_lr_test(m3)
print("\nLikelihood Ratio Test (Model 2 vs Model 3):")
print(f"Chi2: {lrtest_23[0]:.4f}, p-value: {lrtest_23[1]:.4f}, df: {lrtest_23[2]}")

### Portfolio Notebook Improvement Suggestions

- **Clear Narrative & Documentation:**  
  - Add detailed markdown cells explaining the research question, methodology, modeling approach, and key findings.  
  - Provide a project overview, data dictionary, and conclusions.

- **Data Cleaning & Exploration:**  
  - Include exploratory data analysis (EDA) with visualizations (histograms, scatterplots, correlation heatmaps) to show data understanding and cleaning.  
  - Explain how missing values or outliers are handled.

- **Modular and Reproducible Code:**  
  - Organize the code into clear sections (data ingestion, cleaning, transformation, analysis, and reporting) with explanatory comments.  
  - Encapsulate frequently used logic into functions.

- **Error Handling & Validation:**  
  - Introduce error checking, assertions, or informative warnings in your code to improve robustness.

- **Model Evaluation & Interpretation:**  
  - Provide additional model diagnostics such as residual plots, confidence intervals, and goodness-of-fit statistics.  
  - Use summary tables or visualizations to compare different models and explain the improvements.

- **Incorporating Weights:**  
  - Consider incorporating weights in your analysis if your data collection requires it. Explain the weighting scheme and its impact on the results.

- **Interactivity & Presentation:**  
  - Use interactive widgets or visualization libraries (e.g., Plotly, Bokeh) to allow reviewers to engage more closely with your results.  
  - Conclude with a final summary that discusses insights, limitations, and potential future work.